In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Cài nnU-Net bản chuẩn từ PyPI (Nhanh và sạch)
!pip install nnunetv2

# Cài thêm các thư viện bổ trợ nếu thiếu
!pip install nibabel matplotlib medpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 10.4 MB/s eta 0:00:00
  Created wheel for nnunetv2: filename=nnunetv2-2.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for medpy: filename=MedPy-0.5.2-py3-none-any.whl size=224710 sha256=f5265262015ce0c417f2625caf23a75bb6141a46638606fd251fe2255a166fd2
  Stored in directory: /root/.cache/pip/wheels/89/5a/f8/b3def53b9c2133d2f8698ea2173bb5df63bd8e761ce8e9aec9
Successfully built medpy


## 1. Giải nén foler input

In [1]:
!ls

drive  sample_data


### a. Giải nén `nnUNet_preprocessed` vào thư mục content của Colab

In [2]:
import os
import zipfile
from tqdm import tqdm
import shutil

In [3]:
# GIẢI NÉN DỮ LIỆU PREPROCESSED
# ----------------------------------------------------------------------

os.environ['nnUNet_raw'] = "/content/nnUNet_raw"
os.environ['nnUNet_preprocessed'] = "/content/nnUNet_preprocessed"
os.environ['nnUNet_results'] = "/content/drive/MyDrive/XUM_project/nnUNet_results"

!mkdir -p $nnUNet_raw
!mkdir -p $nnUNet_preprocessed

zip_path_drive = "/content/drive/MyDrive/XUM_project/nnUNET_preprocessed.zip"
zip_local = "/content/nnUNET_preprocessed.zip"
extract_root = "/content/nnUNet_preprocessed"

print("📦 Copy file từ Drive…")
!cp "$zip_path_drive" "$zip_local"

print("💨 Copy xong. Bắt đầu giải nén…")

with zipfile.ZipFile(zip_local, 'r') as zip_ref:
    # Detect folder root name inside zip (vd: nnUNET_preprocessed/)
    root_name = zip_ref.namelist()[0].split('/')[0]

    file_list = zip_ref.infolist()
    total_files = len(file_list)

    print(f"🔧 Phát hiện folder root trong ZIP: {root_name}")
    print("🔧 Đang giải nén, loại bỏ lớp folder đầu tiên…")

    for file in tqdm(file_list, total=total_files, desc="🔄 Unzipping"):
        file_path = file.filename

        # Bỏ lớp folder đầu tiên
        fixed_path = file_path.replace(root_name + "/", "", 1)

        if fixed_path.strip() == "":
            continue

        # Chỉ định nơi lưu đúng
        target_path = os.path.join(extract_root, fixed_path)

        # Tạo thư mục nếu cần
        if file.is_dir():
            os.makedirs(target_path, exist_ok=True)
        else:
            os.makedirs(os.path.dirname(target_path), exist_ok=True)
            with zip_ref.open(file) as source, open(target_path, "wb") as target:
                target.write(source.read())

os.remove(zip_local)

print("🎉 GIẢI NÉN XONG VÀ ĐÃ TỰ ĐỘNG FIX FOLDER DOUBLE-LAYER! 😎🔥")

📦 Copy file từ Drive…
💨 Copy xong. Bắt đầu giải nén…
🔧 Phát hiện folder root trong ZIP: nnUNET_preprocessed
🔧 Đang giải nén, loại bỏ lớp folder đầu tiên…


🔄 Unzipping: 100%|██████████| 2585/2585 [03:18<00:00, 13.02it/s]


🎉 GIẢI NÉN XONG VÀ ĐÃ TỰ ĐỘNG FIX FOLDER DOUBLE-LAYER! 😎🔥


### b. Giải nén `nnUNet_raw` vào thư mục content của Colab

In [4]:
# GIẢI NÉN DỮ LIỆU RAW (QUAN TRỌNG CHO INFERENCE)
# ----------------------------------------------------------------------

# 1. Thiết lập đường dẫn (Đảm bảo biến môi trường vẫn đúng)
os.environ['nnUNet_raw'] = "/content/nnUNet_raw"
!mkdir -p $nnUNet_raw

# Đường dẫn file zip trên Drive (Bạn kiểm tra lại tên file cho đúng nhé)
# Giả sử bạn lưu là nnUNet_raw.zip trong thư mục XUM_project
zip_path_drive = "/content/drive/MyDrive/XUM_project/nnUNET_raw.zip"
zip_local = "/content/nnUNet_raw.zip"
extract_root = "/content/nnUNet_raw"

if not os.path.exists(zip_path_drive):
    print(f"❌ Lỗi: Không tìm thấy file {zip_path_drive}")
    print("👉 Hãy kiểm tra lại tên file zip raw trên Google Drive của bạn.")
else:
    print("📦 Copy file Raw Data từ Drive...")
    !cp "$zip_path_drive" "$zip_local"

    print("💨 Copy xong. Bắt đầu giải nén Raw Data...")

    with zipfile.ZipFile(zip_local, 'r') as zip_ref:
        # Detect folder root name inside zip (ví dụ: nnUNet_raw/)
        # Để tránh bị lồng thư mục kiểu /content/nnUNet_raw/nnUNet_raw/Dataset...
        root_name = zip_ref.namelist()[0].split('/')[0]

        file_list = zip_ref.infolist()
        total_files = len(file_list)

        print(f"🔧 Phát hiện folder root trong ZIP: {root_name}")
        print("🔧 Đang giải nén và loại bỏ lớp folder thừa...")

        for file in tqdm(file_list, total=total_files, desc="🔄 Unzipping Raw"):
            file_path = file.filename

            # Logic: Nếu file nằm trong folder root, ta bỏ tên root đi để file nhảy ra ngoài
            # Ví dụ: nnUNet_raw/Dataset101/abc.nii -> Dataset101/abc.nii
            if file_path.startswith(root_name + "/"):
                fixed_path = file_path.replace(root_name + "/", "", 1)
            else:
                fixed_path = file_path # Nếu nén không có folder root thì giữ nguyên

            if fixed_path.strip() == "":
                continue

            # Chỉ định nơi lưu đúng
            target_path = os.path.join(extract_root, fixed_path)

            # Tạo thư mục nếu cần
            if file.is_dir():
                os.makedirs(target_path, exist_ok=True)
            else:
                os.makedirs(os.path.dirname(target_path), exist_ok=True)
                with zip_ref.open(file) as source, open(target_path, "wb") as target:
                    target.write(source.read())

    os.remove(zip_local)
    print("🎉 ĐÃ GIẢI NÉN RAW DATA THÀNH CÔNG! SẴN SÀNG CHẠY INFERENCE! 🚀")

📦 Copy file Raw Data từ Drive...
💨 Copy xong. Bắt đầu giải nén Raw Data...
🔧 Phát hiện folder root trong ZIP: nnUNET_raw
🔧 Đang giải nén và loại bỏ lớp folder thừa...


🔄 Unzipping Raw: 100%|██████████| 1845/1845 [04:28<00:00,  6.87it/s]

🎉 ĐÃ GIẢI NÉN RAW DATA THÀNH CÔNG! SẴN SÀNG CHẠY INFERENCE! 🚀


## 2. Test nhiễu dữ liệu

Thay vì chỉ nói lý thuyết, việc thực hiện một **Thí nghiệm Đối chứng (Controlled Experiment)** bằng cách tác động vào một mẫu (sample) theo hai hướng khác nhau là cách thuyết phục nhất để chứng minh rằng 3 bản đồ bất định của bạn không bị trùng lặp (redundant) mà mang ý nghĩa riêng biệt.

Dưới đây là kế hoạch chi tiết cho thí nghiệm này để bạn đưa vào phần **Qualitative Analysis** (Phân tích định tính):

---

### Mục tiêu Thí nghiệm
Chứng minh khả năng **Phân biệt (Disentanglement)** của mô hình:
1.  Biết phân biệt đâu là **Ảnh xấu/Nhiễu** (cần tăng Aleatoric).
2.  Biết phân biệt đâu là **Ảnh lạ/Dị biệt** (cần tăng Epistemic).

---

### Thiết kế Thí nghiệm: "Một người, Hai số phận"

Chọn **1 lát cắt (slice)** điển hình (có cả khối u và nền rõ ràng). Sau đó tạo ra 2 biến thể từ ảnh gốc này:


In [ ]:
!python /content/drive/MyDrive/XUM_project/src/experiment_noise/main.py --mode edl --val_only --limit 5

📂 Loading Validation set from split file...
📂 Đã load danh sách Validation Fold 0: 74 ca.
🔍 Found 5 cases to process.
Running Noise Experiment:   0% 0/5 [00:00<?, ?it/s]🧪 PROCESSING CASE: BRATS_011

🔥 Running Experiment: Gaussian Noise | Levels: [0.0, 50.0, 100.0, 200.0]
   ⚡ Simulating Gaussian Noise Level = 0.0...
🔧 Initializing Engine | Mode: EDL...
🚀 Initializing nnU-Net Predictor...
📂 Model loaded from: /content/drive/MyDrive/XUM_project/nnUNet_results/Dataset101_BraTS2020/EDLTrainer__nnUNetPlans__3d_fullres

  0% 0/8 [00:00<?, ?it/s]
 12% 1/8 [00:02<00:16,  2.41s/it]
 25% 2/8 [00:04<00:12,  2.09s/it]
 38% 3/8 [00:06<00:10,  2.08s/it]
 50% 4/8 [00:08<00:08,  2.08s/it]
 62% 5/8 [00:10<00:06,  2.09s/it]
 75% 6/8 [00:12<00:04,  2.08s/it]
 88% 7/8 [00:14<00:02,  2.07s/it]
100% 8/8 [00:16<00:00,  2.09s/it]
      📊 Dice: WT=0.92 | TC=0.91 | ET=0.75
      📸 Selected Slice: 110
   ⚡ Simulating Gaussian Noise Level = 50.0...
🔧 Initializing Engine | Mode: EDL...
🚀 Initializing nnU-Net Predi


### **Chiều hướng 2: Tăng Epistemic (Giả lập "Ca bệnh lạ/OOD")**
* **Cách làm:** Tạo ra một dị vật nhân tạo hoặc thay đổi cấu trúc ảnh một cách phi lý.
    * *Cách hay nhất:* Vẽ một **hình hộp màu trắng/xám** đè lên một vùng não lành. Hoặc **lật ngược** ảnh (nếu model chưa từng học ảnh lật).
* **Cơ chế:**
    * Đây là một mẫu dữ liệu "từ trên trời rơi xuống" (Out-of-Distribution). Trong tập train không bao giờ có cái hộp vuông nào nằm trong não cả.
    * Mô hình sẽ rơi vào trạng thái "thiếu kiến thức" hoàn toàn.
* **Kỳ vọng kết quả:**
    * **Epistemic Map:** Sáng rực lên (Tăng đột biến) tại vị trí cái hộp vuông đó.
    * **Aleatoric Map:** Có thể thấp hoặc nhiễu ngẫu nhiên (vì mô hình không tìm thấy biên giới nào quen thuộc để mà phân vân).
* **Thông điệp:** *"Mô hình báo hiệu: Cái này lạ quá, tôi chưa học bao giờ. Đây có thể là dị vật, khối u lạ hoặc lỗi kỹ thuật. Bác sĩ hãy kiểm tra ngay!"*

In [6]:
!python /content/drive/MyDrive/XUM_project/src/experiment_ood/main.py --mode edl --val_only --limit 5

📂 Loading Validation set from split file...
📂 Đã load danh sách Validation Fold 0: 74 ca.
🔍 Found 5 cases to process.
Running OOD Experiment:   0% 0/5 [00:00<?, ?it/s]🧪 OOD EXPERIMENT (TARGETED): BRATS_011

🔥 Running OOD Group: Artificial Artifact
   ⚡ Simulating: Box_White...
🔧 Initializing Engine | Mode: EDL...
🚀 Initializing nnU-Net Predictor...
📂 Model loaded from: /content/drive/MyDrive/XUM_project/nnUNet_results/Dataset101_BraTS2020/EDLTrainer__nnUNetPlans__3d_fullres
🔍 DEBUG: Đang tìm GT cho BRATS_011...
   - Thử: /content/nnUNet_raw/Dataset101_BraTS2020/labelsTr/BRATS_011.nii.gz -> KHÔNG
   - Thử: /content/nnUNet_raw/Dataset101_BraTS2020/labelsTr/BRATS_011.nii -> CÓ

  0% 0/8 [00:00<?, ?it/s]
 12% 1/8 [00:02<00:16,  2.39s/it]
 25% 2/8 [00:04<00:12,  2.09s/it]
 38% 3/8 [00:06<00:10,  2.07s/it]
 50% 4/8 [00:08<00:08,  2.08s/it]
 62% 5/8 [00:10<00:06,  2.08s/it]
 75% 6/8 [00:12<00:04,  2.07s/it]
 88% 7/8 [00:14<00:02,  2.07s/it]
100% 8/8 [00:16<00:00,  2.08s/it]
   ⚡ Simulating: B

In [ ]:
!python /content/drive/MyDrive/XUM_project/src/experiment_ood/main.py --mode edl --val_only --limit 5

📂 Loading Validation set from split file...
📂 Đã load danh sách Validation Fold 0: 74 ca.
🔍 Found 5 cases to process.
Running OOD Experiment:   0% 0/5 [00:00<?, ?it/s]🧪 OOD EXPERIMENT (TARGETED): BRATS_011

🔥 Running OOD Group: Artificial Artifact
   ⚡ Simulating: Box_White...
🔧 Initializing Engine | Mode: EDL...
🚀 Initializing nnU-Net Predictor...
📂 Model loaded from: /content/drive/MyDrive/XUM_project/nnUNet_results/Dataset101_BraTS2020/EDLTrainer__nnUNetPlans__3d_fullres
🔍 DEBUG: Đang tìm GT cho BRATS_011...
   - Thử: /content/nnUNet_raw/Dataset101_BraTS2020/labelsTr/BRATS_011.nii.gz -> KHÔNG
   - Thử: /content/nnUNet_raw/Dataset101_BraTS2020/labelsTr/BRATS_011.nii -> CÓ

  0% 0/8 [00:00<?, ?it/s]
 12% 1/8 [00:02<00:16,  2.41s/it]
 25% 2/8 [00:04<00:12,  2.08s/it]
 38% 3/8 [00:06<00:10,  2.05s/it]
 50% 4/8 [00:08<00:08,  2.05s/it]
 62% 5/8 [00:10<00:06,  2.06s/it]
 75% 6/8 [00:12<00:04,  2.05s/it]
 88% 7/8 [00:14<00:02,  2.04s/it]
100% 8/8 [00:16<00:00,  2.06s/it]
   ⚡ Simulating: B

## 3. Thử kiểm tra giá trị pixel ảnh (xem range để biết thêm dị vật chuẩn không - độ sáng của dị vật nên trong range với pixel ảnh - vì pixel hiện giờ đã qua tiền xử lý)

In [ ]:
import nibabel as nib
import numpy as np
import os

BASE_CONFIG = {
    # Input Data
    "image_folder":     "/content/nnUNet_raw/Dataset101_BraTS2020/imagesTr",
    "label_folder":     "/content/nnUNet_raw/Dataset101_BraTS2020/labelsTr",
    "split_file":       "/content/nnUNet_preprocessed/Dataset101_BraTS2020/splits_final.json", # File JSON chứa danh sách train/val split theo fold.

    # Feature Toggles
    "save_2d_snapshot": True, # Có lưu ảnh chụp lát cắt 2D (file .png) để xem nhanh không.
    "save_3d_nifti":    True, # Có lưu file kết quả 3D (.nii.gz) (Tốn dung lượng nhưng cần để phân tích AUSE), gồm ground_truth, mri_crop, prediction, 3 uncertainty (total/aleatoric/epistemic)
    "calc_metrics":     True, # Có tính toán các chỉ số Dice/HD95 không

    # Metrics Settings
    "metrics_per_class": True, # Tính điểm riêng cho từng vùng u. Với mỗi ca bệnh, code sẽ tính điểm Dice/HD95 cho từng lớp riêng biệt: WT (Whole Tumor), TC (Tumor Core), và ET (Enhancing Tumor)
    "metrics_average":   True,

    # Run Settings
    "run_mode":         "validation_split", # "validation_split" - chạy full tập test | "range" - lấy bao nhiêu ca ra chạy thử | "random" - lấy ngẫu nhiên bao nhiêu ca
    "fold":             0,
    "test_range":       [0, 5],
    "num_random":       5,
    "show_on_screen":   False,
    "figsize":          (30, 6),

    # Output File Names
    "dir_nifti":        "3d_nifti",
    "file_csv_detail":  "metrics_detailed.csv", # Các cột chi tiết như Dice_WT, Dice_TC, Dice_ET cho từng bệnh nhân.
    "file_csv_summary": "metrics_summary.csv", # Tóm tắt trung bình, độ lệch chuẩn cho các lớp (trung bình trên các samples)

    # --- CẤU HÌNH AUSE MỚI ---
    # Giới hạn trục X của biểu đồ Risk-Coverage
    # [Start, End]: 1.0 là giữ 100%, 0.05 là giữ 5%
    "ause_retention_range": [1.0, 0.5],
    "ause_steps": 20, # Độ mịn của biểu đồ (tính toán tại 20 điểm mốc).

# --- [NEW] CẤU HÌNH THÍ NGHIỆM STRESS TEST ---
    "noise_temp_dir": "/content/temp_noise_experiment",

    "noise_experiment_config": {
        # Chọn 1 trong 3 loại để chạy: "Gaussian Noise", "Gaussian Blur", "Motion Ghost"
        "active_type": "Gaussian Blur",

        # Định nghĩa các mức độ cho từng loại
        "settings": {
            "Gaussian Noise": [0.0, 50.0, 100.0, 200.0], # Sigma (Nhiễu hạt)
            "Gaussian Blur":  [0.0, 0.5, 1.0, 1.5],      # Sigma (Làm mờ - Sát thủ Dice)
            "Motion Ghost":   [0, 2, 3, 4]               # Số lượng bóng ma (Ảo ảnh)
        }
    },

    # --- CẤU HÌNH THÍ NGHIỆM OOD (MỚI) ---
    "ood_temp_dir": "/content/temp_ood_experiment",

    "ood_experiment_config": {
        # Chọn loại muốn chạy (hoặc "All" để chạy hết)
        "active_type": "All",

        "settings": {
            # 1. Dị vật (Giả lập khối u lạ hoặc kim loại)
            "Artificial Artifact": ["Box_White", "Box_Grey", "Sphere_White"],

            # 2. Biến dạng cấu trúc (Sai giải phẫu)
            "Structural Mutation": ["Flip_Horizontal", "Flip_Vertical"],

            # 3. Sai lệch cường độ (Máy chụp bị lỗi sáng/tối)
            "Intensity Shift": [0.1, 0.5, 2.0, 5.0]
        }
    }
}

def check_image_intensity():
    print("🕵️‍♂️ KIỂM TRA GIÁ TRỊ PIXEL ẢNH MRI")

    img_dir = BASE_CONFIG["image_folder"]
    # Tìm file đầu tiên
    files = sorted([f for f in os.listdir(img_dir) if f.endswith("_0000.nii.gz") or f.endswith("_0000.nii")])

    if not files:
        print("❌ Không tìm thấy ảnh nào.")
        return

    target_file = os.path.join(img_dir, files[0])
    print(f"📂 Đang đọc file: {target_file}")

    # Load ảnh
    img = nib.load(target_file)
    data = img.get_fdata()

    print("-" * 40)
    print(f"📐 Kích thước (Shape): {data.shape}")
    print(f"🔢 Kiểu dữ liệu (Dtype): {data.dtype}")
    print(f"⬇️ Giá trị Nhỏ nhất (Min): {data.min():.2f}")
    print(f"⬆️ Giá trị Lớn nhất (Max): {data.max():.2f}")
    print(f"ea️ Giá trị Trung bình (Mean): {data.mean():.2f}")
    print("-" * 40)

    # In thử một vùng 5x5 ở giữa não (để tránh vùng nền đen = 0)
    center_slice = data.shape[2] // 2
    cx, cy = data.shape[0] // 2, data.shape[1] // 2
    sample_patch = data[cx:cx+5, cy:cy+5, center_slice]

    print("🔍 Mẫu dữ liệu thực tế (5x5 pixels tại tâm):")
    print(sample_patch)
    print("-" * 40)

    if data.max() > 255:
        print("✅ Kết luận: Đây là ảnh 16-bit/12-bit (High Dynamic Range).")
    else:
        print("⚠️ Kết luận: Đây là ảnh đã được nén về 8-bit.")

if __name__ == "__main__":
    check_image_intensity()

🕵️‍♂️ KIỂM TRA GIÁ TRỊ PIXEL ẢNH MRI
📂 Đang đọc file: /content/nnUNet_raw/Dataset101_BraTS2020/imagesTr/BRATS_001_0000.nii
----------------------------------------
📐 Kích thước (Shape): (240, 240, 155)
🔢 Kiểu dữ liệu (Dtype): float64
⬇️ Giá trị Nhỏ nhất (Min): 0.00
⬆️ Giá trị Lớn nhất (Max): 625.00
ea️ Giá trị Trung bình (Mean): 26.02
----------------------------------------
🔍 Mẫu dữ liệu thực tế (5x5 pixels tại tâm):
[[333. 326. 284. 262. 300.]
 [318. 332. 315. 282. 299.]
 [326. 331. 325. 315. 318.]
 [318. 324. 324. 328. 320.]
 [342. 333. 322. 311. 287.]]
----------------------------------------
✅ Kết luận: Đây là ảnh 16-bit/12-bit (High Dynamic Range).
